# Simulación de un sistema de partículas bajo la influencia del potencial de Lennard-Jones

En el presente trabajo se emplea el algoritmo de Verlet para simular un sistema de partículas como lo podría ser un gas neutro. Para ello se mostrará en una caja de simulación el movimiento de las partículas 

## Importación de Librerías

Si estás ejecutando esto localmente, necesitas tener instalado los siguientes paquetes:

- NumPy
- Numba
- Matplotlib
- Simpy
- IPython

Ejecuta el siguiente comando: `pip install numpy numba matplotlib sympy ipython` en caso de que no los tengas.

In [1]:
%pip install numpy numba matplotlib sympy ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np                          # Para usar operaciones matemáticas
import numba
from numba import jit                       # Optimizador
import matplotlib.pyplot as plt             # Obtención de gráficas
import matplotlib.animation as animation    # Animaciones
import sympy
from sympy import symbols, diff, latex      # Para realizar cálculo simbólico
from IPython.display import display         # Para mostrar en LaTeX
from IPython.core.display import Math
from mpl_toolkits.mplot3d import Axes3D

## Definición de variables

In [3]:
numero_particulas = 20
masa = 2.0    # En UMA
carga = 0.0   # Neutras

# Iniciamos el sistema en una posición aleatoria, con velocidad cero y aceleración cero
# Generamos matrices de dimensión "número de particulas" x "3" (20 x 3 en este caso)
posicion_inicial = np.random.rand(numero_particulas , 3)    # Los valores serán números al azar entre 0 y 1. El 3 indica las 3 dimensiones x, y, z
velocidad_inicial = np.zeros((numero_particulas , 3))       # Velocidades iniciales = 0
aceleracion_inicial = np.zeros((numero_particulas , 3))     # Aceleraciones inicial = 0

El potencial de interacción a emplear en este modelo es el potencial de Lennard-Jones:

$$
V(r)=4 \epsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]
$$

Donde: $r$ es la distancia entre partículas, $\epsilon$ es la profundidad del pozo de potencial y $\sigma$ es la distancia a la cual el potencial se anula.

<p align="center"><img src="https://www.periodni.com/gallery/lennard-jones_potential.png" height="300"></p>

Al ser un potencial conservativo (solo depende de la posición), podemos obtener la fuerza mediante:

$$\vec{F}(r) = - \vec{\nabla} V(r)$$

In [4]:
#Calculamos la fuerza 
#Para ello definimos nuestras variables
r = sympy.Symbol('r')
epsilon = sympy.Symbol('epsilon')
sigma = sympy.Symbol('sigma')

#Definimos el potencial
potencial = 4*epsilon*((sigma/r)**12 - (sigma/r)**6)

#Derivamos para obtener la fuerza 
fuerza = -sympy.diff(potencial, r)

#Mostramos la expresión
display(fuerza)

-4*epsilon*(6*sigma**6/r**7 - 12*sigma**12/r**13)

In [5]:
#Convertimos a LaTeX
expresion_latex = latex(fuerza)

display(Math(r'$F(r) = %s$' % expresion_latex))

<IPython.core.display.Math object>

Reescribimos la expresión para la fuerza:
$$F(r) = - 4 \epsilon \left(\frac{6 \sigma^{6}}{r^{7}} - \frac{12 \sigma^{12}}{r^{13}}\right)$$
Reordenamos y colocamos el vector dirección:
$$\vec{F}(r) = 4 \epsilon \left(12 \frac{\sigma^{12}}{r^{13}} - 6 \frac{\sigma^{6}}{r^{7}}  \right) \hat{r}$$ 

# Algoritmo de Verlet

El fin principal de este trabajo es emplear el algoritmo de Verlet para obtener la simulación.
- El algoritmo de Verlet actualiza la posición, la velocidad y la aceleración mediante iteraciones en el tiempo.
- Para no complicarnos con las características del potencial vamos a considerar como unitarias a las constantes. Sin embargo, si se desea simular algún gas en específico se pueden variar los valores de $\epsilon$ y $\sigma$.
- Considerar que cada par de partículas experimenta una fuerza, entonces se debe redefinir la fuerza para que se aplicada a todas las partículas del sistema:


 $$\vec{F}_{ij}(r) = 4 \epsilon \left(12 \frac{\sigma^{12}}{r_{ij}^{13}} - 6 \frac{\sigma^{6}}{r_{ij}^{7}}  \right) \hat{r}_{ij}$$

In [6]:
#Parámetros del potencial
epsilon = 1.0
sigma = 1.0
umbral = 2.5*sigma          #Por razones computacionales. En este valor el potencial alcanza el mínimo
                            #Cuando se excede este valor el potencial deja de afectar a las partículas, usaremos esto para determinar
                            #a partir de dónde el potencial se anula

#Definimos los parámetros de la simulación
longitud_caja = 10.0

#Pasos de tiempo
dt = 0.01
pasos = 100

### Cálculo de la fuerza

In [12]:
#Definimos una función para poder calcular la fuerza:
def fuerza_ij(r, epsilon, sigma):
    norma_r = np.linalg.norm(r)     #Calculamos la norma de r, es decir, la distancia
    if norma_r > umbral:            #Si la distancia es mayor al umbral, el potencial se anula
        return np.zeros_like(r)     #Devuelve al vector r como un vector nulo
    else:
        f = -4*epsilon*(6*sigma**6/norma_r**7 - 12*sigma**12/norma_r**13)
        return f * r/norma_r        #r/norma_r es el vector unitario de r

### La energía como parámetro de control
En toda la simulación la energía debe conservarse, por ello utilizar esta ley nos permite monitorizar que no haya nada extraño ocurriendo en la simulación.
- Consideremos que solo hay energía en el sistema por el movimiento de las partículas y por el potencial de Lennard-Jones.
- Se calcula entonces la energía cinética.
- Se calcula también la energía potencial.

In [8]:
#Cálculo de la energía cinética

def energia_cinetica(velocidades, masas):
    velocidades_cuadrado = np.sum(velocidades**2) #np.sum(velocities**2, axis=1)
    energia_cinetica = 0.5 * np.sum(masas * velocidades_cuadrado)
    return energia_cinetica

In [9]:
#Cálculo de la energía potencial
@numba.njit
def energia_potencial(posiciones):
    energia = 0.0
    for i in range(numero_particulas):
        for j in range(i+1, numero_particulas):
            r = posiciones[i,:] - posiciones[j,:]
            r = r - longitud_caja * np.round(r / longitud_caja)
            energia += 4.0 * epsilon * ((sigma / np.linalg.norm(r))**12 - (sigma / np.linalg.norm(r))**6)

### Cálculo de la aceleración

In [10]:
@numba.njit
def aceleracion(posicion):
    aceleracion_ij = np.zeros(3)                            #La aceleración es un vector tridimensional
    for i in range(numero_particulas):                      #Iteramos desde la primera partículas hasta la n-ésima
        if i != j:                                          #Para no obtener aceleraciones tipo a_11
            for j in range(numero_particulas):
                r = posicion[j] - posicion[i]               #Calculamos el vector po
                fuerza_escalar = fuerza_ij(r,epsilon, sigma)
                aceleracion_ij[i,j] = fuerza_escalar/masa
                aceleracion_ij[j,i] = - fuerza_escalar/masa #3ra Ley de Newton
    return np.sum(aceleracion_ij)

## Simulación

In [13]:
def simulacion(posicion, velocidad, aceleracion):
    energia_cinetica = np.zeros(pasos)

    for t in range(pasos):
        velocidad_media = velocidad + 0.5 * aceleracion * dt
        posicion = (posicion + velocidad_media * dt) % longitud_caja

        delta_r = posicion[:, np.newaxis] - posicion[np.newaxis, :]
        delta_r = delta_r - longitud_caja * np.round(delta_r / longitud_caja)
        r = np.sqrt(np.sum(delta_r**2, axis=2))

        fuerza = np.zeros((numero_particulas, 3))
        for i in range(numero_particulas):
            for j in range(numero_particulas):
                if i != j:
                    fuerza_ij = fuerza_ij(delta_r[i,j,:], epsilon, sigma)
                    fuerza[i,:] += fuerza_ij
                    fuerza[j,:] -= fuerza_ij

        aceleracion = fuerza / masa

        velocidad = velocidad_media + 0.5 * aceleracion * dt

        energia_cinetica[t] = energia_cinetica(velocidad, masa)

    potencial_energy = energia_potencial(posicion)

    return posicion, velocidad, aceleracion, energia_cinetica, potencial_energy

posicion, velocidad, aceleracion = posicion_inicial, velocidad_inicial, aceleracion_inicial
posicion, velocidad, aceleracion, energia_cinetica, potencial_energy = simulacion(posicion, velocidad, aceleracion)

UnboundLocalError: ignored